In [17]:
from sklearn.externals import joblib

print("Starting evaluation")
print("Loading pipeline")
pipeline = joblib.load("pipeline.pkl")
print("pipeline loaded")

Starting evaluation
Loading pipeline
pipeline loaded


Get evaluation data

In [18]:
import pandas as pd
test_df = pd.read_csv("test_data.csv")
X = test_df.iloc[:,1:]
y = test_df.iloc[:,0]

y_pred = pipeline.predict(X)



In [65]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
cm = confusion_matrix(y, y_pred)
accuracy = accuracy_score(y, y_pred)

print("Test results\n-------------------------------------------------------------\n")
results = {"Accuracy":accuracy, "Confusion matrix":cm.tolist()}
print("Accuracy: {0}%\n".format(accuracy))
print("Confusion matrix:")
print(cm)

Test results
-------------------------------------------------------------

Accuracy: 0.878095238095238%

Confusion matrix:
[[799   0   3   5   6   5  11   2  13   8]
 [  0 911   4   6   0  12   4   2  10   1]
 [ 13   5 722  15  10   2   7   9  12   6]
 [  9  17  42 704   1  47  10  11  19  22]
 [  1   8   5   0 701   2  11   2  11  70]
 [ 16  16  11  18  20 618  16   2  10  20]
 [  9   5   8   0  14  11 772   0  12   0]
 [  7   6  19   1   8   5   0 798   3  37]
 [  3  36  28   8  15  22   6   3 640  41]
 [  9   5  10   5  41  11   1  38   9 711]]


In [83]:
import json

foundResults = False

from pathlib import Path
if Path('results.json').exists():
    with open('results.json', 'r') as f:
        previousResults = json.load(f)
    foundResults = True
else:
    print("Warning, did not find any previous results!!")

In [76]:
def compareResults(Results, PreviousResults):
    print("Comparing results")
    if(Results['Accuracy'] >= PreviousResults['Accuracy']):
        print("\nModel passed the test.")
        return True
    else:
        print("Model did not pass the test.")
        return False
    
def modelPassed():
    #Save the results as the current model
    with open('results.json', 'w') as f:
        json.dump(results, f, ensure_ascii=False, sort_keys=True, indent = 4)
    #Save the results in version control
    with open('./Versions/Pipeline_' + version +'_results.json', 'w') as f:
        json.dump(results, f, ensure_ascii=False, sort_keys=True, indent = 4)

def modelFailed():  
    print("Saving results..")
    with open('./Versions/Failed/Pipeline_' + version +'_results.json', 'w') as f:
        json.dump(results, f, ensure_ascii=False, sort_keys=True, indent = 4)
    print("Results saved.")
    import sys
    sys.exit(1)

In [80]:
import datetime
version = str(datetime.datetime.now())

if foundResults:
    if compareResults(results, previousResults):
        modelPassed()
    else:
        modelFailed()
else:
    print("Deploying model without previous results.")
    modelPassed()

Deploying model without previous results.

Model passed the test.
